In [ ]:
!pip install langchain
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "발급받은 API 키"

In [ ]:
import os

os.environ["GOOGLE_CSE_ID"] = "Google PSE ID"
os.environ["GOOGLE_API_KEY"] = "Google API Key"

In [ ]:
from langchain.chat_models import ChatOpenAI
from llama_index.core import VectorStoreIndex
from langchain.agents import initialize_agent, Tool
from langchain.chains import LLMChain
from langchain import LLMMathChain
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from llama_index.core import download_loader
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import PythonREPL
from langchain.prompts import PromptTemplate
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

llm = ChatOpenAI(temperature=0)

# 데이터베이스 도구
sql_uri = "sqlite:///user_support.db"
db = SQLDatabase.from_uri(sql_uri)
template = """
Given an input question, first refer table_info and create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here. in Korean"

Only use the following tables:
{table_info}

If someone asks for 사용자, 고객, customers, they really mean the users table.
If someone asks for 구매 이력, they really mean the order_history table.
If someone asks for 대응 이력, they really mean the support_history table.

Question: {input}
"""
prompt = PromptTemplate(input_variables=["input", "table_info", "dialect"], template=template)
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=prompt, verbose=True)

# Google Search 도구
search = GoogleSearchAPIWrapper()

# 범용적으로 일반적인 대화를 하기 위해 단일 체인을 생성해 도구로 등록
prompt = PromptTemplate(input_variables=["query"], template="{query} 이 요청문을 가능한 한 한국어로 답변해주세요.")
chain_general = LLMChain(llm=ChatOpenAI(temperature=0.5), prompt=prompt)

# 계산
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

# 파이썬 도구
python_repl = PythonREPL()

# Agent 도구
tools = [
    Tool(
        name="SQL Database",
        func=db_chain.run,
        description="고객에 관련된 것을 데이터베이스에 질의하고 답변할 때 편리합니다.",
    ),
    Tool(
        name = "Google Search",
        func=search.run,
        description="최신 정보를 조사할 때 편리합니다.",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="계산할 때 편리합니다."
    ),
    Tool(
        name="python_repl",
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
        func=python_repl.run
    ),
    Tool(
        name="general",
        func=chain_general.run,
        description="잡담이나 범용적인 질문에 답할 때 편리합니다.",
        return_direct=True,
    ),
]

# 챗봇 Agent 초기화
llm = ChatOpenAI(temperature=0.5)

agent_kwargs = {
    "suffix": """
    시작! 지금부터 대화는 전부 한국어로 진행됩니다.

    이전 대화 이력
    {chat_history}

    새로운 입력: {input}
    {agent_scratchpad}
    """,
}

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, agent_kwargs=agent_kwargs, verbose=True, memory=memory)

## 대화 예

In [ ]:
agent.run("윤 태진님의 정보와 사용자 ID를 알려주세요. 한국어에서는 성, 이름 순서로 이름을 부릅니다.")

In [ ]:
agent.run("윤 태진님과의 사이에 발생한 문제가 있나요? 대응 이력을 확인해주세요.")

In [ ]:
agent.run("오늘 마포구 상암동의 날씨는? 온도는 섭씨로 알려주세요.")

In [ ]:
agent.run("오늘 상암동 날씨가 꽤 춥네요. 그렇게 생각하지 않나요?")

In [ ]:
agent.run("데이터베이스를 참조해서 구매 이력에서 제품별 전체 판매 금액을 계산해주세요. 금액 단위는 대한민국 원입니다.")

In [ ]:
agent.run("제품별 전체 판매 금액을 Python으로 matplotlib을 이용하여 그래프로 그려주세요. 매상이 있는 제품만을 대상으로 하고, 문자 표시는 문자가 깨지지 않도록 영어로 해주세요.")

In [ ]:
agent.run("저는 좀 더 작은 집을 직접 지어, 2년동안 살았던 적이 있습니다. 그리고 지금은 일반적인 삶을 살고 있습니다. 어떻게 생각하시나요?")